# ตัวอย่าง model

In [ ]:
import os
from django.db import models
from django.contrib.auth.models import User
from django.utils import timezone

# โมเดลเสริม (1) สำหรับ OneToOneField
class UserProfile(models.Model):
    user = models.OneToOneField(User, on_delete=models.CASCADE, related_name="profile")
    bio = models.TextField(blank=True)
    profile_pic = models.ImageField(upload_to='profile_pics/', blank=True, null=True)

    def __str__(self):
        return self.user.username

# โมเดลเสริม (2) สำหรับ ManyToManyField
class Tag(models.Model):
    name = models.CharField(max_length=50, unique=True)

    def __str__(self):
        return self.name

# โมเดลหลัก "Kitchen Sink" ของเรา
class ComprehensiveRecord(models.Model):
    # --- Text Fields ---
    title = models.CharField(max_length=200, help_text="หัวข้อหลัก")
    description = models.TextField(blank=True, help_text="รายละเอียดแบบยาว")

    # --- Number Fields ---
    item_count = models.IntegerField(default=0, help_text="จำนวนเต็ม")
    temperature = models.FloatField(null=True, blank=True, help_text="เลขทศนิยม (ไม่แม่นยำสูง)")
    price = models.DecimalField(max_digits=10, decimal_places=2, help_text="เลขทศนิยม (แม่นยำสูง)")

    # --- Boolean Field ---
    is_published = models.BooleanField(default=False, help_text="สถานะเผยแพร่ (True/False)")

    # --- Date & Time Fields ---
    publish_date = models.DateField(null=True, blank=True, help_text="วันที่เผยแพร่ (เฉพาะวัน)")
    publish_time = models.TimeField(null=True, blank=True, help_text="เวลาที่เผยแพร่ (เฉพาะเวลา)")
    last_modified = models.DateTimeField(auto_now=True, help_text="แก้ไขล่าสุด (อัตโนมัติ)")
    created_at = models.DateTimeField(auto_now_add=True, help_text="สร้างเมื่อ (อัตโนมัติครั้งเดียว)")

    # --- Specialized Text Fields ---
    contact_email = models.EmailField(blank=True)
    website = models.URLField(blank=True)
    
    # --- File Fields (ต้องตั้งค่า MEDIA_ROOT) ---
    attachment = models.FileField(upload_to='attachments/', blank=True, null=True)

    # --- Relationship Fields ---
    # OneToOne ถูกลิงก์มาจาก UserProfile
    owner = models.ForeignKey(UserProfile, on_delete=models.CASCADE, related_name="records")
    tags = models.ManyToManyField(Tag, blank=True, related_name="records")

    def __str__(self):
        return self.title

# crud

## setup (คือไรไม่รู้)

In [ ]:
# --- SETUP ---
from yourapp.models import UserProfile, Tag, ComprehensiveRecord
from django.contrib.auth.models import User
from datetime import date, time
from decimal import Decimal
from django.utils import timezone

# 1. สร้าง User และ UserProfile (สำหรับ ForeignKey & OneToOneField)
# (สมมติว่ามี user ที่ username='testuser' อยู่แล้ว ถ้าไม่มีให้สร้างก่อน)
test_user, created = User.objects.get_or_create(username='testuser')
profile, created = UserProfile.objects.get_or_create(user=test_user, defaults={'bio': 'This is a test profile.'})

# 2. สร้าง Tags (สำหรับ ManyToManyField)
# วิธียัดข้อมูลสำหรับ many to many
tag1, created = Tag.objects.get_or_create(name='Python')
tag2, created = Tag.objects.get_or_create(name='Django')
tag3, created = Tag.objects.get_or_create(name='Tutorial')

print("Setup Complete!")

## create (c)

In [ ]:
#### **C - CREATE (การสร้าง)**

# --- CREATE ---

# สร้างอ็อบเจกต์ด้วย .create()
# **สังเกตชนิดข้อมูลที่ส่งเข้าไปในแต่ละ Field**
rec1 = ComprehensiveRecord.objects.create(
    # Text
    title="My First Comprehensive Record",
    description="This record includes various field types.",
    
    # Numbers
    item_count=10,
    temperature=36.6,
    price=Decimal("199.99"),  # **สำคัญมาก: DecimalField ต้องใช้ Decimal หรือ string**

    # Boolean
    is_published=True,

    # Date & Time (แบบกำหนดเอง)
    publish_date=date(2024, 1, 15),
    publish_time=time(10, 30, 0),

    # Specialized Text
    contact_email="test@example.com",
    website="https://djangoproject.com",

    # File (ใน shell เราจะใส่แค่ path ที่สมมติขึ้น)
    attachment="path/to/my/file.pdf",

    # Relationship (ForeignKey)
    # **สำคัญ: ต้องส่ง "อ็อบเจกต์" เข้าไป ไม่ใช่ ID**
    owner=profile 
)

# **สำคัญ: ManyToManyField ไม่สามารถเพิ่มตอน .create() ได้ ต้องทำหลังสร้างเสร็จ**
rec1.tags.add(tag1, tag2)
# .tags คือ ตัวแปรที่อยู่ใน class นะ ถ้าเป็นชื่ออื่นก็เปลี่ยนตาม

print(f"Created Record: '{rec1.title}' with ID: {rec1.id}")
print(f"Tags added: {[tag.name for tag in rec1.tags.all()]}")

## read (r)

In [ ]:
# --- READ ---

# 1. Get: อ่านแบบเจาะจง (ถ้าไม่เจอจะ error)
record = ComprehensiveRecord.objects.get(id=1)
print(f"GET >> Title: {record.title}, Price: {record.price}")

# 2. All: อ่านทั้งหมด
all_records = ComprehensiveRecord.objects.all()
print(f"ALL >> Found {all_records.count()} records.")

# 3. Filter: กรองด้วยเงื่อนไขต่างๆ
#   - ค้นหาจาก BooleanField
published_records = ComprehensiveRecord.objects.filter(is_published=True)

#   - ค้นหาจาก NumberField (ใช้ __gt, __lt, __gte, __lte)
expensive_records = ComprehensiveRecord.objects.filter(price__gte=Decimal("100.00"))

#   - ค้นหาจาก DateField (ใช้ __year, __month, __day)
records_from_2024 = ComprehensiveRecord.objects.filter(publish_date__year=2024)

#   - ค้นหาจาก CharField (ใช้ __icontains, __startswith)
records_with_first = ComprehensiveRecord.objects.filter(title__icontains="first")

#   - ค้นหาข้ามความสัมพันธ์ (ใช้ __)
#     หา Record ทั้งหมดที่เจ้าของ (owner) คือ 'testuser'
records_by_user = ComprehensiveRecord.objects.filter(owner__user__username='testuser')

#     หา Record ทั้งหมดที่มี Tag ชื่อ 'Python'
python_records = ComprehensiveRecord.objects.filter(tags__name='Python')

print(f"FILTER >> Found {python_records.count()} records with 'Python' tag.")

## update (u)

In [ ]:
# --- UPDATE ---

# 1. อ่านอ็อบเจกต์ที่ต้องการแก้ไขออกมาก่อน
rec_to_update = ComprehensiveRecord.objects.get(id=1)
print(f"BEFORE UPDATE >> Title: '{rec_to_update.title}', Stock: {rec_to_update.item_count}")

# 2. แก้ไขค่าในแต่ละ Field
rec_to_update.title = "My Updated Record Title" # CharField
rec_to_update.is_published = False             # BooleanField
rec_to_update.item_count += 5                  # IntegerField
rec_to_update.publish_date = None              # DateField (ทำให้เป็นค่าว่าง)

# 3. สั่งบันทึกการเปลี่ยนแปลง
rec_to_update.save()

print(f"AFTER UPDATE >> Title: '{rec_to_update.title}', Stock: {rec_to_update.item_count}")

# --- การแก้ไข ManyToManyField ---
#   - การลบ Tag
rec_to_update.tags.remove(tag1)
print(f"TAGS AFTER REMOVE >> {[tag.name for tag in rec_to_update.tags.all()]}")

#   - การเพิ่ม Tag ใหม่
rec_to_update.tags.add(tag3)
print(f"TAGS AFTER ADD >> {[tag.name for tag in rec_to_update.tags.all()]}")

#   - การเคลียร์ทั้งหมดแล้วตั้งค่าใหม่ (set)
rec_to_update.tags.set([tag1, tag3])
print(f"TAGS AFTER SET >> {[tag.name for tag in rec_to_update.tags.all()]}")

## delete (d)

In [ ]:
# --- DELETE ---

# 1. อ่านอ็อบเจกต์ที่ต้องการลบ
rec_to_delete = ComprehensiveRecord.objects.get(id=1)
print(f"Preparing to delete record with ID: {rec_to_delete.id}")

# 2. สั่งลบ
result = rec_to_delete.delete()
print(f"Deletion result: {result}")

# 3. ตรวจสอบว่ามันหายไปจริง
try:
    ComprehensiveRecord.objects.get(id=1)
except ComprehensiveRecord.DoesNotExist:
    print("Record with ID 1 no longer exists. Deletion successful.")